In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
constant_patch = '11.23.1'

# 라이엇 개발자 페이지에서 최신 버전 한국 서버 챔피언 정보 가져오기
champion_info = requests.get("http://ddragon.leagueoflegends.com/cdn/" + constant_patch + "/data/ko_KR/champion.json").json()     

In [3]:
# 챔피언 정보 => DataFrame으로 변환
# champion_df는 'key'로 id값, 'name'으로 챔피언 한국어 이름을 가지는 156rows짜리 DataFrame
champion_df = pd.DataFrame(champion_info['data']).T[['key', 'name']]

# champion_df의 'key' column string => numeric 변환
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [4]:
df = pd.read_csv('BIPA_data.csv', index_col=0)

In [5]:
df = df.merge(champion_df, how = 'left', left_on = 'championId', right_on = 'key')

In [6]:
data = pd.DataFrame()
data = df[['tierRank', 'position', 'name', 'result']]

In [7]:
# 결측치 드랍
data = data.dropna(axis=0)
# 다시하기 드랍
data.drop(data.loc[data['result']=='UNKNOWN'].index, inplace=True)
print('data : ', len(data))

data :  963577


In [8]:
# result 숫자로 변환
data.loc[data['result'] == 'WIN', 'result'] = 1
data.loc[data['result'] == 'LOSE', 'result'] = 0
data['games'] = 1
data = data.astype({'result' : 'int'})
# result -> wins로 컬럼명 변경
data.rename(columns = {'result':'wins'},inplace=True)

In [9]:
# 티어별 데이터 프레임 생성

IRON = data[data['tierRank'].str.contains('I')]
print('IRON : ', len(IRON))
BRONZE = data[data['tierRank'].str.contains('B')]
print('BRONZE : ', len(BRONZE))
SILVER = data[data['tierRank'].str.contains('S')]
print('SILVER : ', len(SILVER))
GOLD = data[data['tierRank'].str.contains('G')]
print('GOLD : ', len(GOLD))
PLATINUM = data[data['tierRank'].str.contains('P')]
print('PLATINUM : ', len(PLATINUM))
DIAMOND = data[data['tierRank'].str.contains('D')]
print('DIAMOND : ', len(DIAMOND))
m_list = ['M', 'R', 'C']
m_list = '|'.join(m_list)
ABOVE_MASTER = data[data['tierRank'].str.contains(m_list)]
print('MASTER+ : ', len(ABOVE_MASTER))
# GRAND_MASTER = data[data['tierRank'].str.contains('R')]
# print('GRAND_MASTER : ', len(GRAND_MASTER))
# CHALLENGER = data[data['tierRank'].str.contains('C')]
# print('CHALLENGER : ', len(CHALLENGER))
print('SUM : ', len(IRON)+len(BRONZE)+len(SILVER)+len(GOLD)+len(PLATINUM)+len(DIAMOND)+len(ABOVE_MASTER))

IRON :  9857
BRONZE :  125597
SILVER :  344653
GOLD :  328557
PLATINUM :  124452
DIAMOND :  26585
MASTER+ :  3876
SUM :  963577


In [10]:
IRON_total = IRON.groupby(['position', 'name'], as_index=False).sum()
IRON_sum = IRON.groupby(['position'], as_index=False).sum()
IRON_sum.columns = ['position', 'wins', 'total_games', 'tier']
IRON_sum = IRON_sum[['position', 'total_games']]
IRON_total = IRON_total.merge(IRON_sum, how = 'left', on = 'position')
IRON_total = IRON_total[IRON_total.games > 50]
IRON_total['winRate'] = IRON_total['wins'] / IRON_total['games']
IRON_total['pickRate'] = IRON_total['games'] / IRON_total['total_games']
IRON_total['']
IRON_total = IRON_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

BRONZE_total = BRONZE.groupby(['position', 'name'], as_index=False).sum()
BRONZE_sum = BRONZE.groupby(['position'], as_index=False).sum()
BRONZE_sum.columns = ['position', 'wins', 'total_games']
BRONZE_sum = BRONZE_sum[['position', 'total_games']]
BRONZE_total = BRONZE_total.merge(BRONZE_sum, how = 'left', on = 'position')
BRONZE_total = BRONZE_total[BRONZE_total.games > 50]
BRONZE_total['winRate'] = BRONZE_total['wins'] / BRONZE_total['games']
BRONZE_total['pickRate'] = BRONZE_total['games'] / BRONZE_total['total_games']
BRONZE_total = BRONZE_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

SILVER_total = SILVER.groupby(['position', 'name'], as_index=False).sum()
SILVER_sum = SILVER.groupby(['position'], as_index=False).sum()
SILVER_sum.columns = ['position', 'wins', 'total_games']
SILVER_sum = SILVER_sum[['position', 'total_games']]
SILVER_total = SILVER_total.merge(SILVER_sum, how = 'left', on = 'position')
SILVER_total = SILVER_total[SILVER_total.games > 50]
SILVER_total['winRate'] = SILVER_total['wins'] / SILVER_total['games']
SILVER_total['pickRate'] = SILVER_total['games'] / SILVER_total['total_games']
SILVER_total = SILVER_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

GOLD_total = GOLD.groupby(['position', 'name'], as_index=False).sum()
GOLD_sum = GOLD.groupby(['position'], as_index=False).sum()
GOLD_sum.columns = ['position', 'wins', 'total_games']
GOLD_sum = GOLD_sum[['position', 'total_games']]
GOLD_total = GOLD_total.merge(GOLD_sum, how = 'left', on = 'position')
GOLD_total = GOLD_total[GOLD_total.games > 50]
GOLD_total['winRate'] = GOLD_total['wins'] / GOLD_total['games']
GOLD_total['pickRate'] = GOLD_total['games'] / GOLD_total['total_games']
GOLD_total = GOLD_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

PLATINUM_total = PLATINUM.groupby(['position', 'name'], as_index=False).sum()
PLATINUM_sum = PLATINUM.groupby(['position'], as_index=False).sum()
PLATINUM_sum.columns = ['position', 'wins', 'total_games']
PLATINUM_sum = PLATINUM_sum[['position', 'total_games']]
PLATINUM_total = PLATINUM_total.merge(PLATINUM_sum, how = 'left', on = 'position')
PLATINUM_total = PLATINUM_total[PLATINUM_total.games > 50]
PLATINUM_total['winRate'] = PLATINUM_total['wins'] / PLATINUM_total['games']
PLATINUM_total['pickRate'] = PLATINUM_total['games'] / PLATINUM_total['total_games']
PLATINUM_total = PLATINUM_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

DIAMOND_total = DIAMOND.groupby(['position', 'name'], as_index=False).sum()
DIAMOND_sum = DIAMOND.groupby(['position'], as_index=False).sum()
DIAMOND_sum.columns = ['position', 'wins', 'total_games']
DIAMOND_sum = DIAMOND_sum[['position', 'total_games']]
DIAMOND_total = DIAMOND_total.merge(DIAMOND_sum, how = 'left', on = 'position')
DIAMOND_total = DIAMOND_total[DIAMOND_total.games > 50]
DIAMOND_total['winRate'] = DIAMOND_total['wins'] / DIAMOND_total['games']
DIAMOND_total['pickRate'] = DIAMOND_total['games'] / DIAMOND_total['total_games']
DIAMOND_total = DIAMOND_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

ABOVE_MASTER_total = ABOVE_MASTER.groupby(['position', 'name'], as_index=False).sum()
ABOVE_MASTER_sum = ABOVE_MASTER.groupby(['position'], as_index=False).sum()
ABOVE_MASTER_sum.columns = ['position', 'wins', 'total_games']
ABOVE_MASTER_sum = ABOVE_MASTER_sum[['position', 'total_games']]
ABOVE_MASTER_total = ABOVE_MASTER_total.merge(ABOVE_MASTER_sum, how = 'left', on = 'position')
ABOVE_MASTER_total = ABOVE_MASTER_total[ABOVE_MASTER_total.games > 50]
ABOVE_MASTER_total['winRate'] = ABOVE_MASTER_total['wins'] / ABOVE_MASTER_total['games']
ABOVE_MASTER_total['pickRate'] = ABOVE_MASTER_total['games'] / ABOVE_MASTER_total['total_games']
ABOVE_MASTER_total = ABOVE_MASTER_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

In [11]:
IRON_total

,position,name,wins,games,total_games,winRate,pickRate
409,T,말파이트,37,69,1885,0.536232,0.036605
410,T,모데카이저,37,69,1885,0.536232,0.036605
428,T,세트,126,251,1885,0.501992,0.133156
393,T,다리우스,46,94,1885,0.489362,0.049867
383,T,가렌,45,94,1885,0.478723,0.049867
399,T,레넥톤,39,84,1885,0.464286,0.044562
445,T,오른,37,80,1885,0.462500,0.042440
439,T,아트록스,33,74,1885,0.445946,0.039257
424,T,사일러스,27,64,1885,0.421875,0.033952
453,T,이렐리아,35,84,1885,0.416667,0.044562


In [12]:
BRONZE_total

,position,name,wins,games,total_games,winRate,pickRate
700,T,클레드,66,98,25412,0.673469,0.003856
598,T,릴리아,59,95,25412,0.621053,0.003738
600,T,마오카이,59,96,25412,0.614583,0.003778
688,T,카시오페아,41,70,25412,0.585714,0.002755
655,T,오공,153,272,25412,0.562500,0.010704
...,...,...,...,...,...,...,...
23,A,루시안,337,734,24016,0.459128,0.030563
110,A,칼리스타,140,305,24016,0.459016,0.012700
14,A,드레이븐,59,131,24016,0.450382,0.005455
52,A,세나,32,77,24016,0.415584,0.003206


In [13]:
SILVER_total

,position,name,wins,games,total_games,winRate,pickRate
650,T,세주아니,33,52,69195,0.634615,0.000751
604,T,드레이븐,31,51,69195,0.607843,0.000737
656,T,스웨인,34,58,69195,0.586207,0.000838
593,T,그레이브즈,82,143,69195,0.573427,0.002067
696,T,자크,152,274,69195,0.554745,0.003960
...,...,...,...,...,...,...,...
93,A,자야,319,686,68922,0.465015,0.009953
22,A,루시안,934,2019,68922,0.462605,0.029294
89,A,이렐리아,69,153,68922,0.450980,0.002220
124,A,킨드레드,48,112,68922,0.428571,0.001625


In [14]:
GOLD_total

,position,name,wins,games,total_games,winRate,pickRate
637,T,쉬바나,62,101,65392,0.613861,0.001545
582,T,녹턴,37,61,65392,0.606557,0.000933
664,T,올라프,42,71,65392,0.591549,0.001086
693,T,카시오페아,153,259,65392,0.590734,0.003961
607,T,말자하,46,78,65392,0.589744,0.001193
...,...,...,...,...,...,...,...
23,A,루시안,791,1720,66908,0.459884,0.025707
64,A,신드라,73,160,66908,0.456250,0.002391
55,A,세나,59,132,66908,0.446970,0.001973
85,A,요네,30,70,66908,0.428571,0.001046


In [15]:
PLATINUM_total

,position,name,wins,games,total_games,winRate,pickRate
593,T,일라오이,103,172,24693,0.598837,0.006966
548,T,사이온,106,183,24693,0.579235,0.007411
626,T,탈론,41,72,24693,0.569444,0.002916
588,T,우르곳,86,152,24693,0.565789,0.006156
496,T,갱플랭크,139,250,24693,0.556000,0.010124
...,...,...,...,...,...,...,...
27,A,미스 포츈,206,438,25185,0.470320,0.017391
92,A,케이틀린,509,1097,25185,0.463993,0.043558
28,A,바루스,254,559,25185,0.454383,0.022196
72,A,자야,142,318,25185,0.446541,0.012627


In [24]:
DIAMOND_total

,position,name,wins,games,total_games,winRate,pickRate
372,T,렝가,33,52,5353,0.634615,0.009714
381,T,마오카이,49,81,5353,0.604938,0.015132
450,T,케넨,61,105,5353,0.580952,0.019615
421,T,오공,58,104,5353,0.557692,0.019428
430,T,이렐리아,182,331,5353,0.549849,0.061834
...,...,...,...,...,...,...,...
11,A,루시안,43,89,5348,0.483146,0.016642
48,A,징크스,109,230,5348,0.473913,0.043007
29,A,아펠리오스,232,517,5348,0.448743,0.096672
69,A,트위치,22,51,5348,0.431373,0.009536


In [17]:
ABOVE_MASTER_Top = ABOVE_MASTER_total[ABOVE_MASTER_total['position'] == 'T'].sort_values(by=['pickRate'], ascending=False).head(20)

In [18]:
ABOVE_MASTER_total[ABOVE_MASTER_total['position'] == 'T'].sort_values(by=['pickRate'], ascending=False).head(20)

,position,name,wins,games,total_games,winRate,pickRate
259,T,카밀,30,67,737,0.447761,0.090909
254,T,제이스,27,55,737,0.490909,0.074627


In [27]:
DIAMOND_total[DIAMOND_total['position'] == 'S'].sort_values(by=['winRate'], ascending=False)

,position,name,wins,games,total_games,winRate,pickRate
284,S,마오카이,131,217,5289,0.603687,0.041029
344,S,트런들,86,147,5289,0.585034,0.027794
306,S,소나,65,113,5289,0.575221,0.021365
303,S,세나,66,117,5289,0.564103,0.022121
307,S,소라카,44,79,5289,0.556962,0.014937
275,S,렐,34,62,5289,0.548387,0.011722
334,S,질리언,44,83,5289,0.530120,0.015693
274,S,레오나,222,424,5289,0.523585,0.080166
296,S,블리츠크랭크,151,289,5289,0.522491,0.054642
266,S,노틸러스,119,229,5289,0.519651,0.043297


In [ ]:
I = IRON_total[IRON_total['position'] == 'T'].sort_values(by=['games'], ascending=True)

In [ ]:
sns.displot(data = GOLD_Top, x= 'winRate' ,hue = 'position')

In [ ]:
IRON_total['games'].quantile(0.50)

In [ ]:
GOLD_total['games'].quantile(0.30)

In [25]:
TOP_all = pd.concat([IRON_total[IRON_total['position'] == 'T'], BRONZE_total[BRONZE_total['position'] == 'T']])

In [26]:
TOP_all

,position,name,wins,games,total_games,winRate,pickRate
409,T,말파이트,37,69,1885,0.536232,0.036605
410,T,모데카이저,37,69,1885,0.536232,0.036605
428,T,세트,126,251,1885,0.501992,0.133156
393,T,다리우스,46,94,1885,0.489362,0.049867
383,T,가렌,45,94,1885,0.478723,0.049867
...,...,...,...,...,...,...,...
583,T,라이즈,75,169,25412,0.443787,0.006650
595,T,리 신,66,150,25412,0.440000,0.005903
623,T,샤코,23,53,25412,0.433962,0.002086
713,T,판테온,61,145,25412,0.420690,0.005706


In [28]:
data = pd.read_csv('BIPA_data.csv', index_col=0)

In [36]:
data

,gameId,createDate,tierRank,position,teamId,summonerId,championId,result,level,championsKilled,...,totalDamageTaken,neutralMinionsKilledEnemyJungle,neutralMinionsKilledTeamJungle,visionScore,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,lane,gameLength,version
0,5383880006,2021-08-11 06:59:31,P318,S,200,3210877,99,WIN,10,4,...,3602,0,0,11,2830,2181,2181,NONE,972,11.16.390.1945
1,5383880006,2021-08-11 06:59:31,P466,S,100,7550211,35,LOSE,8,1,...,5077,0,2,24,2013,0,0,NONE,972,11.16.390.1945
2,5383880006,2021-08-11 06:59:31,P329,M,200,9306696,266,WIN,11,5,...,11029,0,0,15,6158,585,585,NONE,972,11.16.390.1945
3,5383880006,2021-08-11 06:59:31,P461,T,200,20281103,92,WIN,11,8,...,9944,0,0,10,7615,2318,2318,NONE,972,11.16.390.1945
4,5383880006,2021-08-11 06:59:31,P30,A,200,26082075,22,WIN,10,3,...,6387,0,0,13,3218,2370,2370,NONE,972,11.16.390.1945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5384659633,2021-08-11 17:20:44,S146,S,100,21560397,53,WIN,15,1,...,32961,0,0,63,34725,1978,126,BOTTOM,2379,11.16.390.1945
999996,5384659633,2021-08-11 17:20:44,G442,M,100,46320531,166,WIN,18,10,...,24674,1,16,16,21371,8406,7042,MIDDLE,2379,11.16.390.1945
999997,5384659633,2021-08-11 17:20:44,G418,J,200,64820251,64,LOSE,17,5,...,45911,6,68,22,62426,20292,454,JUNGLE,2379,11.16.390.1945
999998,5384659633,2021-08-11 17:20:44,S178,A,100,72310857,81,WIN,17,6,...,28187,8,9,17,19664,11090,1062,BOTTOM,2379,11.16.390.1945


In [58]:
value = data['championId'].value_counts()

In [59]:
value

81     37562
64     26741
22     23101
39     22244
875    20172
       ...  
77       782
136      632
42       621
427      592
72       422
Name: championId, Length: 156, dtype: int64

In [61]:
champId = value.index

In [66]:
champId = champId.sort_values( ascending=False) 

In [32]:
data.columns

Index(['gameId', 'createDate', 'tierRank', 'position', 'teamId', 'summonerId',
       'championId', 'result', 'level', 'championsKilled', 'numDeaths',
       'assists', 'neutralMinionsKilled', 'turretsKilled', 'barracksKilled',
       'minionsKilled', 'goldEarned', 'physicalDamageDealtToChampions',
       'magicDamageDealtPlayer', 'physicalDamageTaken',
       'sightWardsBoughtInGame', 'visionWardsBoughtInGame', 'wardKilled',
       'wardPlaced', 'totalHeal', 'totalDamageDealtToChampions',
       'totalDamageDealt', 'totalDamageTaken',
       'neutralMinionsKilledEnemyJungle', 'neutralMinionsKilledTeamJungle',
       'visionScore', 'damageSelfMitigated', 'damageDealtToObjectives',
       'damageDealtToTurrets', 'lane', 'gameLength', 'version'],
      dtype='object')